In [221]:
#imports 

import numpy as np
import pandas as pd 
import seaborn as sns 
import openpyxl
import pickle 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import classification_report


In [222]:
df = pd.read_excel('C:\\Users\\Malavi\\Desktop\\RFISUBCODDES Predictor\\Ancillary Rev - 2015 - 2023.xlsx', header=2)
df


,Flight Date,Point of Sale,Channel,RFISUBCOD,RFISUBCODDES,RFICOD,REMARKS,Count,Sum of Revenue USD
0,2015-01-01,Sri Lanka,ATO/CTO/GSA/PSA,0BJ,UPGRADE,Air transportation,UPGRADE,2,1223.78
1,2015-01-01,Sri Lanka,Call centre,0BJ,UPGRADE,Air transportation,UPGRADE,61,13516.32
2,2015-01-01,Hong Kong,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,4,384.23
3,2015-01-01,India Tamilnadu - Chennai,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,26,1658.70
4,2015-01-01,India Trivandrum,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,6,272.64
...,...,...,...,...,...,...,...,...,...
149997,2020-01-07,India Northern,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,2,4.06
149998,2020-01-07,Singapore,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,12.60
149999,2020-01-07,Sri Lanka,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,1,3.04
150000,2020-01-07,United Kingdom,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,25.08


In [223]:
df.isnull().sum()


Flight Date             0
Point of Sale           0
Channel                 0
RFISUBCOD               0
RFISUBCODDES            0
RFICOD                  0
REMARKS               688
Count                   0
Sum of Revenue USD      1
dtype: int64

In [224]:
df = df.dropna()
df.shape

(149313, 9)

In [225]:
df.isnull().sum()

Flight Date           0
Point of Sale         0
Channel               0
RFISUBCOD             0
RFISUBCODDES          0
RFICOD                0
REMARKS               0
Count                 0
Sum of Revenue USD    0
dtype: int64

In [226]:
# Convert Flight Date to datetime and extract Year & Month


df['Flight Date'] = pd.to_datetime(df['Flight Date'])
df['Year'] = df['Flight Date'].dt.year
df['Month'] = df['Flight Date'].dt.month



C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\2572032901.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Flight Date'] = pd.to_datetime(df['Flight Date'])
C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\2572032901.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['Flight Date'].dt.year
C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\2572032901.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [227]:
df.tail(5)

,Flight Date,Point of Sale,Channel,RFISUBCOD,RFISUBCODDES,RFICOD,REMARKS,Count,Sum of Revenue USD,Year,Month
149997,2020-01-07,India Northern,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,2,4.06,2020,1
149998,2020-01-07,Singapore,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,12.60,2020,1
149999,2020-01-07,Sri Lanka,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,1,3.04,2020,1
150000,2020-01-07,United Kingdom,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,25.08,2020,1
150001,2020-01-07,India Northern,ATO/CTO/GSA/PSA,0IJ,EXCESS PIECE,Baggage,EXCESS WEIGHT,3,98.95,2020,1


In [228]:
df = df.drop(['REMARKS', 'RFICOD','Channel','Flight Date'], axis=1)

In [229]:
df.head()

,Point of Sale,RFISUBCOD,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month
0,Sri Lanka,0BJ,UPGRADE,2,1223.78,2015,1
1,Sri Lanka,0BJ,UPGRADE,61,13516.32,2015,1
2,Hong Kong,0IK,EXCESS WEIGHT,4,384.23,2015,1
3,India Tamilnadu - Chennai,0IK,EXCESS WEIGHT,26,1658.70,2015,1
4,India Trivandrum,0IK,EXCESS WEIGHT,6,272.64,2015,1


In [230]:
df = df.drop('RFISUBCOD', axis=1)

In [231]:
df.head()

,Point of Sale,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month
0,Sri Lanka,UPGRADE,2,1223.78,2015,1
1,Sri Lanka,UPGRADE,61,13516.32,2015,1
2,Hong Kong,EXCESS WEIGHT,4,384.23,2015,1
3,India Tamilnadu - Chennai,EXCESS WEIGHT,26,1658.70,2015,1
4,India Trivandrum,EXCESS WEIGHT,6,272.64,2015,1


In [232]:
df.tail()

,Point of Sale,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month
149997,India Northern,CARBON OFFSET,2,4.06,2020,1
149998,Singapore,CARBON OFFSET,5,12.60,2020,1
149999,Sri Lanka,CARBON OFFSET,1,3.04,2020,1
150000,United Kingdom,CARBON OFFSET,5,25.08,2020,1
150001,India Northern,EXCESS PIECE,3,98.95,2020,1


In [233]:
df.isnull().sum()

Point of Sale         0
RFISUBCODDES          0
Count                 0
Sum of Revenue USD    0
Year                  0
Month                 0
dtype: int64

In [234]:
df.to_csv('cleaned_data.csv', index=False)


In [235]:
df.describe()

,Count,Sum of Revenue USD,Year,Month
count,149313.000000,149313.000000,149313.000000,149313.000000
mean,3.920449,347.589336,2017.671924,6.747202
std,6.963465,1212.119752,1.302187,3.551077
min,1.000000,0.000000,2015.000000,1.000000
25%,1.000000,29.780000,2017.000000,4.000000
50%,2.000000,83.030000,2018.000000,7.000000
75%,4.000000,244.900000,2019.000000,10.000000
max,193.000000,46906.530000,2020.000000,12.000000


In [236]:
#Compute revenue per transaction
df['Revenue_per_Count'] = df['Sum of Revenue USD'] / df['Count']

#Find index of max Revenue_per_Count per group
idx = df.groupby(['Point of Sale', 'Month', 'Year'])['Revenue_per_Count'].idxmax()

#Create a new column and assign best ancillary only at those idx positions
df['Best_Ancillary'] = np.nan
df.loc[idx, 'Best_Ancillary'] = df.loc[idx, 'RFISUBCODDES']

#Forward-fill and back-fill within each group to populate the rest
df['Best_Ancillary'] = df.groupby(['Point of Sale', 'Month', 'Year'])['Best_Ancillary'].ffill().bfill()

C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\2676873235.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EXCESS WEIGHT' 'EXCESS PIECE' 'UPGRADE' ...
 'PRE-RESERVED SEAT ASSIGNMENT' 'PRE PAID BAGGAGE' 'PRE PAID BAGGAGE']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'Best_Ancillary'] = df.loc[idx, 'RFISUBCODDES']


In [237]:
df.isnull().sum()

Point of Sale         0
RFISUBCODDES          0
Count                 0
Sum of Revenue USD    0
Year                  0
Month                 0
Revenue_per_Count     0
Best_Ancillary        0
dtype: int64

In [238]:
df.head(5)

,Point of Sale,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month,Revenue_per_Count,Best_Ancillary
0,Sri Lanka,UPGRADE,2,1223.78,2015,1,611.890000,EXCESS WEIGHT
1,Sri Lanka,UPGRADE,61,13516.32,2015,1,221.579016,EXCESS WEIGHT
2,Hong Kong,EXCESS WEIGHT,4,384.23,2015,1,96.057500,EXCESS WEIGHT
3,India Tamilnadu - Chennai,EXCESS WEIGHT,26,1658.70,2015,1,63.796154,EXCESS WEIGHT
4,India Trivandrum,EXCESS WEIGHT,6,272.64,2015,1,45.440000,EXCESS WEIGHT


In [239]:
df.tail(5)

,Point of Sale,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month,Revenue_per_Count,Best_Ancillary
149997,India Northern,CARBON OFFSET,2,4.06,2020,1,2.030000,EXCESS PIECE
149998,Singapore,CARBON OFFSET,5,12.60,2020,1,2.520000,EXCESS WEIGHT
149999,Sri Lanka,CARBON OFFSET,1,3.04,2020,1,3.040000,UPGRADE
150000,United Kingdom,CARBON OFFSET,5,25.08,2020,1,5.016000,UPGRADE
150001,India Northern,EXCESS PIECE,3,98.95,2020,1,32.983333,EXCESS PIECE


In [245]:
value_counts = df['Best_Ancillary'].value_counts()

In [246]:
frequent_values = value_counts[value_counts >= 10].index

In [ ]:
df_fil = df[df['Best_Ancillary'].isin(frequent_values)]

In [250]:
df_fil['Best_Ancillary'].value_counts()

Best_Ancillary
UPGRADE                           59663
PRE PAID BAGGAGE                  29096
EXCESS PIECE                      18793
EXCESS WEIGHT                     18214
TRANSPORTATION CREDIT VOUCHER      4027
DEPOSITS/DOWN PAYMENTS             3235
PENALTY FEE                        3175
PRE-RESERVED SEAT ASSIGNMENT       2346
OXYGEN                             2243
UPTO20KG - BAGGAGE                 1603
UPTO55LB/25KG - BAGGAGE            1536
UPTO40KG - BAGGAGE                 1406
EXCESS BAGGAGE TRADER TRAFFIC       923
PET IN HOLD                         688
STRETCHER                           617
UPTO110LB/50KG ¿ BAGGAGE            427
PLUSGRADE                           420
UPTO66LB/30KG - BAGGAGE             274
PENALTY FEE FOR PARTIAL GROUPS      271
SPECIAL CAKE                        217
UPTO22LB/10KG - BAGGAGE             122
CARBON OFFSET                        16
Name: count, dtype: int64

In [256]:
df.shape

(149313, 11)

In [257]:
df_fil.shape

(149312, 11)

In [254]:
#modeling

le_pos = LabelEncoder()
le_anc = LabelEncoder()
le_Banc = LabelEncoder()

df_fil['Point_of_Sale_encoded'] = le_pos.fit_transform(df_fil['Point of Sale'])
df_fil['RFISUBCODDES_encoded'] = le_anc.fit_transform(df_fil['RFISUBCODDES'])
df_fil['Best_Ancillary_encoded'] = le_Banc.fit_transform(df_fil['Best_Ancillary'])

C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\426198135.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['Point_of_Sale_encoded'] = le_pos.fit_transform(df_fil['Point of Sale'])
C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\426198135.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['RFISUBCODDES_encoded'] = le_anc.fit_transform(df_fil['RFISUBCODDES'])
C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\426198135.py:9: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [260]:
# Define features and target
X = df_fil[['Point_of_Sale_encoded', 'Year', 'Month', 'Count', 'Revenue_per_Count']]
y = df_fil['Best_Ancillary_encoded']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [261]:
# Initialize XGBoost classifier
model = XGBClassifier(
    objective='multi:softmax',  # for multi-class classification
    num_class=len(le_Banc.classes_),  # number of classes
    eval_metric='mlogloss',  # metric for multi-class
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=22, ...)

In [262]:
# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Generate classification report
print(classification_report(y_test, y_pred, target_names=le_Banc.classes_))

Accuracy: 0.69
                                precision    recall  f1-score   support

                 CARBON OFFSET       1.00      1.00      1.00         2
        DEPOSITS/DOWN PAYMENTS       0.63      0.52      0.57       650
 EXCESS BAGGAGE TRADER TRAFFIC       0.56      0.51      0.53       191
                  EXCESS PIECE       0.65      0.66      0.66      3802
                 EXCESS WEIGHT       0.68      0.62      0.65      3664
                        OXYGEN       0.68      0.52      0.59       438
                   PENALTY FEE       0.70      0.38      0.49       636
PENALTY FEE FOR PARTIAL GROUPS       0.36      0.38      0.37        53
                   PET IN HOLD       0.67      0.58      0.62       134
                     PLUSGRADE       0.39      0.50      0.44        84
              PRE PAID BAGGAGE       0.70      0.61      0.65      5828
  PRE-RESERVED SEAT ASSIGNMENT       0.69      0.33      0.45       492
                  SPECIAL CAKE       0.71      0

In [ ]:
#model save as pickle file
with open('xgboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)